In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

import numpy as np
import pickle

In [2]:
# fix random seed for reproducibility
np.random.seed(7)

with open('embedded_docs.p') as f:
    data, labels = pickle.load(f)

In [4]:
padded_data = sequence.pad_sequences(data, maxlen=200,padding="pre", truncating="post", value=0.0, dtype='float32')
#padded_data = np.zeros((len(data),200,100))

In [6]:
padded_data[-1]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [-0.0883111 ,  0.02037529, -0.11833055, ..., -0.05252652,
        -0.12606385, -0.02482243],
       [ 0.02274016, -0.05844065, -0.04485843, ...,  0.05721832,
         0.02858308, -0.08743977],
       [ 0.00777967,  0.01736618, -0.0182593 , ...,  0.04863099,
         0.04696344, -0.06476068]], dtype=float32)

In [13]:
# load the dataset but only keep the top n words, zero the rest
X_train, X_test, y_train, y_test = train_test_split(padded_data, labels, train_size=0.9, stratify=labels)

In [14]:
model = Sequential()
model.add(LSTM(100, input_shape = (200, 100)))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 80,501
Trainable params: 80,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
13042/13042 [==============================] - 174s - loss: 0.4651 - acc: 0.8543     

In [25]:
# serialize weights to HDF5
model.save_weights("keras_weights.h5")